# Data Engineering - Apache Mxnet

Creating Dataset according to the Apache Mxnet RecordIO structure for processing training 

1. Format required: 

 222 2 5 0.000 0.4 0.3 0.9 0.44  path/to/image11.jpg
 
id, num_cols(2), num_cols(5), label(0.000), xmin, ymin, xmax, ymax, image_path

index header_column label_width className xmin ymin xmax ymax path

Creating copy

In [ ]:
train_df = train.copy()
test_df = test.copy()

Renaming columns

In [ ]:
train_df.rename(columns = {"LabelName": "className"}, inplace = True)
test_df.rename(columns = {"LabelName": "className"}, inplace = True)

Creating header and label width columns

In [ ]:
train_df["header_cols"] = 2
train_df["label_width"] = 5

Creating Image path columns

In [ ]:
train_df["ImagePath"] = "001.Plastic_bag/images/train/"+train_df['ImageID']+ '.jpg'
test_df["ImagePath"] = "001.Plastic_bag/images/test/"+test_df['ImageID']+ '.jpg'

Arranging the dataset according to the RecordIO format

In [ ]:
train_df = train_df[['header_cols', 'label_width', 'className', 'XMin', 'YMin', 'XMax', 'YMax', 'ImagePath']]
test_df = test_df[['header_cols', 'label_width', 'className', 'XMin', 'YMin', 'XMax', 'YMax', 'ImagePath']]

pd.set_option("display.precision",4)

final_train_df = train_df.copy()
final_train_df['className'] = "0.000"

final_test_df = test_df.copy()
final_test_df['className'] = "0.000"

Moving Images to the new path

In [ ]:
import shutil
shutil.move('','')

### Transposing Images: Data augmentation

1. The bounding box will also transpose according to the formula: LeftRightFlip

768 = image width 

xmin = (768/2) - (xmin - (768/2))

xmax = (768/2) - (xmax - (768/2))

2. The bouding box coordinates of only x axis will change

3. Creatin .lst files for production 

In [ ]:
test_images_path = ''
train_images_path = ''



def augment_data(dataset):
    images_path = train_images_path if dataset == "train" else test_images_path if dataset == "test" else 0
    if images_path == 0:
        raise Exception("Invalid Input parameter")
    start = 0
    end = 0
    if dataset == "train":
        start,end,df = 29,45,final_train_df
    elif dataset == "test":
        start,end,df = 28,44,final_test_df
    
    
    temp_df = pd.DataFrame(columns=["header_cols","label_width","className","XMin","YMin","XMax","YMax","ImagePath"],dtype = object)
    counter = 0
    for index,row in df.iterrows():
        img_path = row["ImagePath"]#image path in the df
        id = img_path[start:end]
        im_path = f"/{dataset}/{id}.jpg" #image path for the physical location of the image(full path
        img = Image.open(im_path)
        image_width,image_height = img.size
        img_flip = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
        img_flip.save(f"/{dataset}/flipped_{id}.jpg")
        new_image_path = f"/{dataset}/flipped_{id}.jpg"
        
        xmin = row['XMin'] * image_width
        xmax = row['XMax'] * image_width
        ymin = row['YMin']
        ymax = row['YMax']
        
        #getting the new coordinates for the flipped bounding boxes
        new_xmin = ((image_width/2)-(xmin-(image_width/2))) / image_width
        new_xmax = ((image_width/2)-(xmax-(image_width/2))) / image_width
        
        
        temp_df.loc[counter]=[2,5,"0.000",new_xmin,ymin,new_xmax,ymax,new_image_path]
        counter +=1
    
    df_merged = df.append(temp_df, ignore_index = True)

    df_merged.to_csv(f"{dataset}.lst", sep = "\t", float_format = "%.4f", header = None)
    print(len(df))#original df
    print("augmented df length below")
    print(len(temp_df))